# Vanilla PPO + action wrapper + reward shaping

Run on 3 environments

Empty Room

Empty Room Random

Four Rooms

In [1]:
import gymnasium as gym
import minigrid
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from minigrid.wrappers import FlatObsWrapper
import time
import numpy as np
from minigrid.core.world_object import Goal
import random
from stable_baselines3.common.callbacks import BaseCallback
from gymnasium.envs.registration import register
from minigrid.core.constants import OBJECT_TO_IDX, IDX_TO_OBJECT, COLOR_TO_IDX, IDX_TO_COLOR, DIR_TO_VEC

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


# First Empty

In [2]:
# Create and wrap the environment
env_id = "MiniGrid-Empty-16x16-v0"

In [3]:
class ActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super(ActionWrapper, self).__init__(env)
        # Define a new action space with only the relevant actions
        self.action_space = gym.spaces.Discrete(3)  # Only three actions: left, right, forward

    def action(self, action):
        # Map the new actions to the original actions
        action_mapping = {
            0: 0,  # left
            1: 1,  # right
            2: 2   # forward
        }
        return action_mapping[action]

In [4]:
class RewardShapingWrapper1(gym.Wrapper):
    def __init__(self, env):
        super(RewardShapingWrapper1, self).__init__(env)
        self.last_action = None
        self.spin_counter = 0  # Tracks consecutive left-right turns

    def step(self, action):
        obs, reward, done, info, extra = self.env.step(action)
        current_pos = self.env.agent_pos

        # Check for spinning behavior
        if self.last_action in [0, 1] and action in [0, 1] and action != self.last_action:
            self.spin_counter += 1
        else:
            self.spin_counter = 0

        if self.spin_counter > 2:  # Threshold for considering it spinning
            reward -= 10
            self.spin_counter = 0  # Reset counter after penalty

        if self.last_action == 0 and action == 1 or self.last_action == 1 and action == 0:
            reward -= 10  # Increase penalty for oscillating between left and right

        self.last_action = action
        return obs, reward, done, info, extra

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        self.last_action = None
        self.spin_counter = 0
        return obs

    def is_facing_wall(self):
        x, y = self.env.agent_pos
        direction_idx = self.env.agent_dir
        delta = self.env.DIR_TO_VEC[direction_idx]
        next_x, next_y = x + delta[0], y + delta[1]
        if 0 <= next_x < self.env.width and 0 <= next_y < self.env.height:
            next_cell = self.env.grid.get(next_x, next_y)
            return next_cell is not None and next_cell.type == 'wall'
        return False
    
    def get_goal_position(self):
        for x in range(self.env.width):
            for y in range(self.env.height):
                if self.env.grid.get(x, y) is not None and isinstance(self.env.grid.get(x, y), Goal):
                    return (x, y)
        return None


In [5]:
class RewardShapingWrapper2(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.last_room_id = None
        self.goal_position = None

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        self.last_room_id = self.get_room_id(obs)
        self.goal_position = self.find_goal_position()
        return obs

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        current_room_id = self.get_room_id(obs)
        agent_pos = self.env.agent_pos

        # Check if the agent has passed through a doorway
        if current_room_id != self.last_room_id:
            reward += 0.1  
            self.last_room_id = current_room_id

        # Check if the goal is reached by comparing positions
        if tuple(agent_pos) == self.goal_position:
            reward += 100 
            terminated = True  

        return obs, reward, terminated, truncated, info

    def get_room_id(self, obs):
        # Implement logic to determine the room ID based on the agent's position
        agent_pos = self.env.agent_pos
        if agent_pos[0] < self.env.width // 2:
            if agent_pos[1] < self.env.height // 2:
                return 0  
            else:
                return 2  
        else:
            if agent_pos[1] < self.env.height // 2:
                return 1  
            else:
                return 3  

    def find_goal_position(self):
        # Scan the grid to find the goal's position
        for x in range(self.env.width):
            for y in range(self.env.height):
                if isinstance(self.env.grid.get(x, y), Goal):
                    return (x, y)
        return None


In [6]:
class RewardShapingWrapper3(gym.Wrapper):
    def __init__(self, env):
        super(RewardShapingWrapper3, self).__init__(env)
        self.last_pos = None
        self.goal_pos = None
        self.stuck_counter = 0

    def step(self, action):
        action = self.noisy_action(action)  # Apply noise to the action for exploration
        obs, reward, done, info, extra = self.env.step(action)
        current_pos = self.env.agent_pos

        if self.goal_pos is None:
            self.goal_pos = self.get_goal_position()

        current_room = self.get_room(current_pos)
        last_room = self.get_room(self.last_pos) if self.last_pos else None

        # Check if transitioned to a new room
        if last_room and current_room and last_room != current_room:
            reward += 5.0  # Reward for moving to a new room

        # Additional vector to the nearest gap or doorway
        nearest_gap_vector = self.get_vector_to_nearest_gap(current_pos)
        obs['nearest_gap_direction'] = nearest_gap_vector  # Ensure obs is a dictionary

        # Assuming obs is a dictionary that includes 'image' and possibly other keys
        goal_direction = self.get_goal_direction(current_pos)
        # If you intend to add goal_direction to obs, you should add it as a new key-value pair
        obs['goal_direction'] = goal_direction  # Add it like this if obs is a dictionary

        # Reward shaping calculations
        if self.last_pos is not None and self.goal_pos is not None:
            distance_reward = self.calculate_reward_shaping(current_pos)
            additional_reward = self.additional_rewards(action, current_pos, self.last_pos)
            reward += distance_reward + additional_reward

        if np.array_equal(current_pos, self.goal_pos):
            reward += 100
            done = True

        if self.last_pos == current_pos:
            self.stuck_counter += 1
        else:
            self.stuck_counter = 0

        if self.stuck_counter > 3:
            obs = self.nudge_agent(current_pos) or obs
            self.stuck_counter = 0

        self.last_pos = current_pos
        return obs, reward, done, info, extra

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        self.last_pos = self.env.agent_pos
        self.goal_pos = self.get_goal_position()
        self.gaps = self.find_gaps()  # Find gaps upon reset
        return obs
    
    def get_vector_to_nearest_gap(self, current_pos):
        if not self.gaps:
            return np.zeros(2)  # Default to zero vector if no gaps identified

        nearest_gap = min(self.gaps, key=lambda gap: np.linalg.norm(np.array(gap) - np.array(current_pos)))
        direction_vector = np.array(nearest_gap) - np.array(current_pos)
        norm = np.linalg.norm(direction_vector)
        return direction_vector / norm if norm != 0 else direction_vector

    
    def find_gaps(self):
        width, height = self.env.width, self.env.height
        gaps = []
        # Typically, gaps will be in the walls that divide the rooms
        # We scan horizontal and vertical mid-lines for empty spaces
        mid_vertical = width // 2
        mid_horizontal = height // 2

        # Vertical mid-line (check for empty cells)
        for y in range(height):
            if self.env.grid.get(mid_vertical, y) is None:
                #print('vertical', self.env.grid.get(mid_vertical, y))
                #print('the gap vertical mid-line is',(mid_vertical, y) )
                gaps.append((mid_vertical, y))

        # Horizontal mid-line (check for empty cells)
        for x in range(width):
            if self.env.grid.get(x, mid_horizontal) is None:
                gaps.append((x, mid_horizontal))

        return gaps


    def get_goal_position(self):
        for x in range(self.env.width):
            for y in range(self.env.height):
                if self.env.grid.get(x, y) is not None and isinstance(self.env.grid.get(x, y), Goal):
                    return (x, y)
        return None
    
    
    def calculate_reward_shaping(self, current_pos):
        if not self.goal_pos:
            return 0
        
        epsilon = 0.01  # Small value to avoid division by zero
        prev_distance = np.linalg.norm(np.array(self.last_pos) - np.array(self.goal_pos))
        current_distance = np.linalg.norm(np.array(current_pos) - np.array(self.goal_pos))
        
        if current_distance < prev_distance:
            return 0.1 * (1 / (current_distance + epsilon))
        elif current_distance > prev_distance:
            return -0.1 * (1 / (current_distance + epsilon))
        return 0

    def additional_rewards(self, action, current_pos, prev_pos):
        prev_distance = np.linalg.norm(np.array(prev_pos) - np.array(self.goal_pos))
        current_distance = np.linalg.norm(np.array(current_pos) - np.array(self.goal_pos))
        distance_change = prev_distance - current_distance
        
        if action in [0, 1] and distance_change <= 0:
            return -0.05  # Unchanged, penalty for non-improvement
        
        reward = 0
        if action == 2 and distance_change > 0:
            reward = min(0.05 * distance_change, 1.0)  # Cap the reward to avoid excessively high values

        return reward
    

    def get_goal_direction(self, current_pos):
        if not self.goal_pos:
            return np.zeros_like(current_pos)  # Return a zero vector if goal position is not set
        # Assuming current_pos and goal_pos are numpy arrays or can be converted to them
        direction_vector = np.array(self.goal_pos) - np.array(current_pos)
        # Normalize the vector
        norm = np.linalg.norm(direction_vector)
        if norm == 0:
            return direction_vector  # Avoid division by zero if already at the goal
        return direction_vector / norm
    
    def noisy_action(self, action):
        # Add random noise to the action occasionally
        if random.random() < 0.1:  # With 10% probability, alter the action randomly
            return random.choice([0, 1, 2])  # Assuming actions are 0, 1, 2
        return action
    

    def get_possible_moves(self, current_pos):
        moves = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Directions: up, down, left, right

        for dx, dy in directions:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)
            if self.is_position_valid(new_pos):
                moves.append(new_pos)
        return moves

    def is_position_valid(self, pos):
        x, y = pos
        # Check if within grid bounds
        if 0 <= x < self.env.width and 0 <= y < self.env.height:
            # Check if the cell is empty (None means empty in MiniGrid)
            return self.env.grid.get(x, y) is None
        return False
    
    def nudge_agent(self, current_pos):
        possible_moves = self.get_possible_moves(current_pos)
        if possible_moves:
            new_pos = random.choice(possible_moves)
            self.env.agent_pos = new_pos  # Assuming you can set the position
            # Simulate a no-op or a neutral action to update state and get observation
            obs, _, _, _, _ = self.env.step(0)  # Consider '0' as a no-op action, if applicable
            return obs
        # If no moves are possible or other issues arise, return a default observation
        return np.zeros(self.env.observation_space.shape)  # Adjust based on the specific obs space
    
    def get_room(self, position):
        x, y = position
        if x < 10 and y < 10:
            return 1  # Top-left room
        elif x >= 10 and y < 10:
            return 2  # Top-right room
        elif x < 10 and y >= 10:
            return 3  # Bottom-left room
        elif x >= 10 and y >= 10:
            return 4  # Bottom-right room
        return 0  # Should not happen unless position is out of expected range

In [7]:
def create_env():
    env = gym.make(env_id, render_mode="rgb_array")#,max_episode_steps=200)
    env = ActionWrapper(env)
    env = RewardShapingWrapper1(env)
    env = FlatObsWrapper(env)  
    return env

# Use the function in make_vec_env
env_vec = make_vec_env(create_env, n_envs=1)
env_vec2 = make_vec_env(create_env, n_envs=4)

In [8]:
model = PPO("MlpPolicy", env_vec2, verbose=1,
            tensorboard_log="log/ppo_reward_empty_no_step_cap"
            )

# Train the model
model.learn(total_timesteps=200000)

Using cpu device
Logging to log/ppo_reward_empty_no_step_cap/PPO_2


/Users/anthony/.pyenv/versions/3.10.13/envs/cs5756-project/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.agent_pos to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.agent_pos` for environment variables or `env.get_wrapper_attr('agent_pos')` that will search the reminding wrappers.
  logger.warn(


----------------------------------
| rollout/           |           |
|    ep_len_mean     | 841       |
|    ep_rew_mean     | -2.07e+03 |
| time/              |           |
|    fps             | 5062      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 8192      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 785         |
|    ep_rew_mean          | -1.68e+03   |
| time/                   |             |
|    fps                  | 2554        |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.012359096 |
|    clip_fraction        | 0.275       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000418    |
|    learning_rate  

In [9]:
model.save("models/ppo_reward_empty_no_step_cap")

In [10]:
model = PPO.load("models/ppo_reward_empty_no_step_cap",env=env_vec)

mean_reward, std_reward = evaluate_policy(model, env=env_vec, n_eval_episodes=50)
print(f"Mean Reward: {mean_reward} +/- {std_reward}")

Mean Reward: 0.9762699999999999 +/- 1.1102230246251565e-16


In [11]:
num_episodes = 50
total_rewards = []  # List to store total rewards for each episode

for episode in range(num_episodes):
    obs = env_vec.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env_vec.step(action)
        #print(done)
        total_reward += reward
        env_vec.render('human')  # Render the environment at each step
        #time.sleep(0.05)  # Adjust this to control the speed of the rendering
    total_rewards.append(total_reward)  # Store the total reward for this episode
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# Calculate the average reward across all episodes
average_reward = sum(total_rewards) / num_episodes
print(f"Average Reward over {num_episodes} episodes: {average_reward}")

env_vec.close()  # Close the environment when done

Episode 1: Total Reward = [0.97626954]
Episode 2: Total Reward = [0.97626954]
Episode 3: Total Reward = [0.97626954]
Episode 4: Total Reward = [0.97626954]
Episode 5: Total Reward = [0.97626954]
Episode 6: Total Reward = [0.97626954]
Episode 7: Total Reward = [0.97626954]
Episode 8: Total Reward = [0.97626954]
Episode 9: Total Reward = [0.97626954]
Episode 10: Total Reward = [0.97626954]
Episode 11: Total Reward = [0.97626954]
Episode 12: Total Reward = [0.97626954]
Episode 13: Total Reward = [0.97626954]
Episode 14: Total Reward = [0.9753906]
Episode 15: Total Reward = [0.9753906]
Episode 16: Total Reward = [0.97451174]
Episode 17: Total Reward = [0.97626954]
Episode 18: Total Reward = [0.97626954]
Episode 19: Total Reward = [0.97626954]
Episode 20: Total Reward = [0.9753906]
Episode 21: Total Reward = [0.97626954]
Episode 22: Total Reward = [0.97626954]
Episode 23: Total Reward = [0.97626954]
Episode 24: Total Reward = [0.9753906]
Episode 25: Total Reward = [0.97626954]
Episode 26: T

# Empty random room

In [24]:
register(
    id='custom_empty-v0',
    entry_point='emptyrandom:RandomGoalEmptyEnv',
    kwargs={}
)

/Users/anthony/.pyenv/versions/3.10.13/envs/cs5756-project/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment custom_empty-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [25]:
#gym.pprint_registry()  # to see all registered environments

if 'custom_empty-v0' in gym.envs.registry:
    print("Environment 'custom_empty-v0' is registered.")
else:
    print("Environment 'custom_empty-v0' is NOT registered.")

Environment 'custom_empty-v0' is registered.


In [26]:
env_id2 = 'custom_empty-v0'
def create_env():
    env = gym.make(env_id2, render_mode="rgb_array",size=16)#,max_episode_steps=200)
    env = ActionWrapper(env)
    env = RewardShapingWrapper2(env)
    env = FlatObsWrapper(env)  
    return env

env_vec_ran = make_vec_env(create_env, n_envs=1)
env_vec_ran2 = make_vec_env(create_env, n_envs=4)

In [27]:
# Initialize the PPO model
model2 = PPO("MlpPolicy", env_vec_ran2, verbose=1,
            tensorboard_log="log/ppo_reward_empty_random_no_step_cap"
            )

# Train the model
model2.learn(total_timesteps=200000)

Using cpu device
Logging to log/ppo_reward_empty_random_no_step_cap/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 518      |
|    ep_rew_mean     | 78.3     |
| time/              |          |
|    fps             | 4858     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 575         |
|    ep_rew_mean          | 77.7        |
| time/                   |             |
|    fps                  | 2892        |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.008512385 |
|    clip_fraction        | 0.0928      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       

: 

In [16]:
model2.save("models/ppo_reward_empty_random_no_step_cap")

In [17]:
model2 = PPO.load("models/ppo_reward_empty_random_no_step_cap",env=env_vec_ran)

mean_reward, std_reward = evaluate_policy(model2, env=env_vec_ran, n_eval_episodes=50)
print(f"Mean Reward: {mean_reward} +/- {std_reward}")

Mean Reward: 99.62877734000001 +/- 10.619204641583398


In [18]:
num_episodes = 50
total_rewards = []  # List to store total rewards for each episode

for episode in range(num_episodes):
    obs = env_vec_ran.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model2.predict(obs)
        obs, reward, done, info = env_vec_ran.step(action)
        #print(done)
        total_reward += reward
        env_vec_ran.render('human')  # Render the environment at each step
        #time.sleep(0.05)  # Adjust this to control the speed of the rendering
    total_rewards.append(total_reward)  # Store the total reward for this episode
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# Calculate the average reward across all episodes
average_reward = sum(total_rewards) / num_episodes
print(f"Average Reward over {num_episodes} episodes: {average_reward}")

env_vec_ran.close()  # Close the environment when done

Episode 1: Total Reward = [100.99648]
Episode 2: Total Reward = [101.27539]
Episode 3: Total Reward = [100.99648]
Episode 4: Total Reward = [101.28242]
Episode 5: Total Reward = [100.99648]
Episode 6: Total Reward = [101.27803]
Episode 7: Total Reward = [100.992966]
Episode 8: Total Reward = [100.99209]
Episode 9: Total Reward = [101.084175]
Episode 10: Total Reward = [101.279785]
Episode 11: Total Reward = [101.07715]
Episode 12: Total Reward = [101.26749]
Episode 13: Total Reward = [101.180664]
Episode 14: Total Reward = [101.28155]
Episode 15: Total Reward = [101.561325]
Episode 16: Total Reward = [101.08594]
Episode 17: Total Reward = [101.18242]
Episode 18: Total Reward = [101.08154]
Episode 19: Total Reward = [101.27276]
Episode 20: Total Reward = [101.089455]
Episode 21: Total Reward = [100.99121]
Episode 22: Total Reward = [101.08242]
Episode 23: Total Reward = [101.17715]
Episode 24: Total Reward = [100.99033]
Episode 25: Total Reward = [110.774414]
Episode 26: Total Reward = 

# Now Four rooms

In [19]:
env_id3 = "MiniGrid-FourRooms-v0"
def create_env():
    env = gym.make(env_id3, render_mode="rgb_array")
    env = ActionWrapper(env)
    env = RewardShapingWrapper3(env)
    env = FlatObsWrapper(env)  
    return env

env_vec_four = make_vec_env(create_env, n_envs=1)
env_vec_four2 = make_vec_env(create_env, n_envs=4)

In [20]:
# Initialize the PPO model
model3 = PPO("MlpPolicy", env_vec_four2, verbose=1,
            tensorboard_log="log/ppo_reward_four_no_step_cap"
            )

# Train the model
model3.learn(total_timesteps=1000000)

Using cpu device
Logging to log/ppo_reward_four_no_step_cap/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 81.4     |
|    ep_rew_mean     | -1.98    |
| time/              |          |
|    fps             | 3207     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 81.9        |
|    ep_rew_mean          | -0.541      |
| time/                   |             |
|    fps                  | 2005        |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.010081182 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    e

In [21]:
model3.save("models/ppo_reward_four_no_step_cap")

In [22]:
model3 = PPO.load("models/ppo_reward_four_no_step_cap",env=env_vec_four)

mean_reward, std_reward = evaluate_policy(model3, env=env_vec_four, n_eval_episodes=50)
print(f"Mean Reward: {mean_reward} +/- {std_reward}")

Mean Reward: 1.3715903600000001 +/- 3.7553126334404747


In [23]:
num_episodes = 50
total_rewards = []  # List to store total rewards for each episode

for episode in range(num_episodes):
    obs = env_vec_four.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model3.predict(obs)
        obs, reward, done, info = env_vec_four.step(action)
        #print(done)
        total_reward += reward
        env_vec_four.render('human')  # Render the environment at each step
        #time.sleep(0.05)  # Adjust this to control the speed of the rendering
    total_rewards.append(total_reward)  # Store the total reward for this episode
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# Calculate the average reward across all episodes
average_reward = sum(total_rewards) / num_episodes
print(f"Average Reward over {num_episodes} episodes: {average_reward}")

env_vec_four.close()  # Close the environment when done

Episode 1: Total Reward = [0.23766619]
Episode 2: Total Reward = [-0.1304092]
Episode 3: Total Reward = [-0.03447063]
Episode 4: Total Reward = [0.76020575]
Episode 5: Total Reward = [0.08032629]
Episode 6: Total Reward = [-0.3582736]
Episode 7: Total Reward = [-0.08206747]
Episode 8: Total Reward = [0.11576329]
Episode 9: Total Reward = [-0.2556608]
Episode 10: Total Reward = [-0.05138635]
Episode 11: Total Reward = [-0.57655513]
Episode 12: Total Reward = [-0.15714954]
Episode 13: Total Reward = [-0.10914353]
Episode 14: Total Reward = [0.23213537]
Episode 15: Total Reward = [0.86677706]
Episode 16: Total Reward = [0.07554837]
Episode 17: Total Reward = [-0.43270564]
Episode 18: Total Reward = [0.31736016]
Episode 19: Total Reward = [-0.03136783]
Episode 20: Total Reward = [-0.2865913]
Episode 21: Total Reward = [19.891245]
Episode 22: Total Reward = [-0.16959403]
Episode 23: Total Reward = [0.02665466]
Episode 24: Total Reward = [-0.48664954]
Episode 25: Total Reward = [-0.4785459]
